<h1>Libraries

In [9]:
import h5py
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import cross_validate, KFold
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict

In [10]:
test_size    = 0.20
seed         = 9
h5_color     = 'C:/Users/zambr/OneDrive/Escritorio/7mo Semestre/Dataset Mineria/histo/color_features2.h5'
h5_colorLb   = 'C:/Users/zambr/OneDrive/Escritorio/7mo Semestre/Dataset Mineria/histo/color_features_labels2.h5'
scoring      = {'accuracy' : make_scorer(accuracy_score),
                'f1_score' : make_scorer(f1_score, average = 'weighted'), 
                'precision' : make_scorer(precision_score, average = 'weighted'),
                'recall' : make_scorer(recall_score, average = 'weighted')}

<h1>Libraries

In [11]:
# import the feature vector and trained labels
h5f_color   = h5py.File(h5_color, 'r')
h5f_colorLb = h5py.File(h5_colorLb, 'r')

color_features_string          = h5f_color['ojosCaracteristicas2']
color_features_labels_string   = h5f_colorLb['ojosCaracteristicas_Etiquetas2']

color_features = np.array(color_features_string)
color_features_labels  = np.array(color_features_labels_string)

h5f_color.close()
h5f_colorLb.close()

# verify the shape of the feature vector and labels
print("[STATUS] features shape: {}".format(color_features.shape))
print("[STATUS] labels shape: {}".format(color_features_labels.shape))

print("[STATUS] training started...")

[STATUS] features shape: (7285, 512)
[STATUS] labels shape: (7285,)
[STATUS] training started...


In [12]:
global_features = np.hstack([color_features])
print("[STATUS] features shape: {}".format(global_features.shape))


[STATUS] features shape: (7285, 512)


<h1>Identify X and Y

In [13]:
X = global_features
Y = color_features_labels

<h1>Training and Testing

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                        test_size   = test_size,
                                        random_state = seed)

X_train.shape, X_test.shape

((5828, 512), (1457, 512))

<h1>Kernels

In [15]:
linear_param_grid = {
    
        "kernel": ["linear"],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

In [16]:
sigmoid_param_grid = {
        "kernel": ["sigmoid"],
        "gamma": ["auto", "scale"],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

<h1>Sigmoid Kernel

In [17]:
sigmoid_grid = GridSearchCV(svm.SVC(), sigmoid_param_grid)
sigmoid_grid.fit(X_train,y_train)
print(sigmoid_grid.best_estimator_)

SVC(C=10, decision_function_shape='ovo', gamma='auto', kernel='sigmoid')


In [18]:
sigmoid_pred = sigmoid_grid.predict(X_test)
sigmoid_accuracy = accuracy_score(y_test, sigmoid_pred)
sigmoid_precision = precision_score(y_test, sigmoid_pred, average='weighted')
sigmoid_recall = recall_score(y_test, sigmoid_pred, average='weighted')
sigmoid_f1 = f1_score(y_test, sigmoid_pred, average='weighted')
print('Accuracy (Sigmoid Kernel): ', "%.2f" % (sigmoid_accuracy*100))
print('F1 (Sigmoid Kernel): ', "%.2f" % (sigmoid_f1*100))
print('Precision (Sigmoid Kernel): ', "%.2f" % (sigmoid_precision*100))
print('Recall (Sigmoid Kernel): ', "%.2f" % (sigmoid_recall*100))

Accuracy (Sigmoid Kernel):  75.57
F1 (Sigmoid Kernel):  75.29
Precision (Sigmoid Kernel):  76.00
Recall (Sigmoid Kernel):  75.57


In [19]:
results = cross_validate(estimator=sigmoid_grid, X=X_train, y=y_train, cv=8, scoring=scoring)

print('Accuracy (Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (Sigmoid Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (Sigmoid Kernel):  76.01
F1 (Sigmoid Kernel):  75.75
Precision (Sigmoid Kernel):  76.61
Recall (Sigmoid Kernel):  76.01


<h1>Linear Kernel


In [20]:
linear_grid = GridSearchCV(svm.SVC(), linear_param_grid)
linear_grid.fit(X_train,y_train)
print(linear_grid.best_estimator_)

SVC(C=0.1, decision_function_shape='ovo', kernel='linear')


In [21]:
linear_pred = linear_grid.predict(X_test)
linear_accuracy = accuracy_score(y_test, linear_pred)
linear_precision = precision_score(y_test, linear_pred, average='weighted')
linear_recall = recall_score(y_test, linear_pred, average='weighted')
linear_f1 = f1_score(y_test, linear_pred, average='weighted')
print('Accuracy (Linear Kernel): ', "%.2f" % (linear_accuracy*100))
print('F1 (Linear Kernel): ', "%.2f" % (linear_f1*100))
print('Precision (Linear Kernel): ', "%.2f" % (linear_precision*100))
print('Recall (Linear Kernel): ', "%.2f" % (linear_recall*100))

Accuracy (Linear Kernel):  76.60
F1 (Linear Kernel):  76.19
Precision (Linear Kernel):  77.51
Recall (Linear Kernel):  76.60


In [22]:
results = cross_validate(estimator=linear_grid, X=X_train, y=y_train, scoring=scoring)

print('Accuracy (Linear Kernel): ', "%.2f" % (np.mean(results['test_accuracy'])*100))
print('F1 (Linear Kernel): ', "%.2f" % (np.mean(results['test_f1_score'])*100))
print('Precision (Linear Kernel): ', "%.2f" % (np.mean(results['test_precision'])*100))
print('Recall (Linear Kernel): ', "%.2f" % (np.mean(results['test_recall'])*100))

Accuracy (Linear Kernel):  76.17
F1 (Linear Kernel):  75.80
Precision (Linear Kernel):  77.09
Recall (Linear Kernel):  76.17
